In [1]:
import pandas as pd
import requests
import string
import time
import datadotworld as dw
import os
import json
from io import StringIO
from pandas.io.json import json_normalize

In [8]:
##CONFIG.PY##
#URL and session variables
url = 'https://mainecampaignfinance.com/api///DataDownload/CSVDownloadReport'
url_base = 'https://mainecampaignfinance.com'
s = requests.Session()
cookies = requests.cookies.RequestsCookieJar()


#Parameters for looping through search

election_years = ['2019','2020']

transaction_types = {'contributions_raw':'CON'
                    ,'expenditures_raw':'EXP'}

trans_types = list(transaction_types.values())
filenames = list(transaction_types.keys())

#Data dictionary for query
data = {}

#POST headers
headers = {'Host':'mainecampaignfinance.com'
            ,'Origin': url_base
            ,'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:69.0) Gecko/20100101 Firefox/69.0'
            ,'Accept': 'application/octet-stream'
            ,'Accept-Language': 'en-US,en;q=0.5'
            ,'Accept-Encoding': 'gzip, deflate, br'
            ,'Referer':'https://mainecampaignfinance.com/'
            ,'Content-Type': 'application/json;charset=utf-8'
            ,'Connection': 'keep-alive'
            ,'TE':'Trailers'
            ,'Pragma': 'no-cache'
            ,'Cache-Control': 'no-cache'}

In [9]:
#GET SPENDING REPORTS#

#Set index
x=0
for b in trans_types:
    
    data.update({"transactionType":trans_types[x]})
    
    #Declare/clear dataframes
    dfs=[]
    df=[]
    
    #Reset index
    i=0
    for n in election_years:

        data.update({"year":election_years[i]})

        r = s.post(url, data=json.dumps(data), headers=headers)
        df = pd.read_csv(StringIO(r.content.decode('utf-8')))
        dfs.append(df)
        i+=1
        
    #Combine dfs
    df=pd.concat(dfs,sort=False,ignore_index=True).drop_duplicates()
    
#     count = dw.query('darrenfishell/2020-maine-state-campaign-finance', 'SELECT COUNT(*) FROM '+filenames[x]).dataframe['count'][0]
#     if count < len(df):
    print('Writing file '+filenames[x]+'.csv to data.world, at '+str(len(df))+' records.')
    with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance',filenames[x]+'.csv') as w:
        df.to_csv(w,index=False)
    
    #Advance to next transaction type
    x+=1

Writing file contributions_raw.csv to data.world, at 15871 records.
Writing file expenditures_raw.csv to data.world, at 6233 records.


In [57]:
r = s.post(url, data=json.dumps(data), headers=headers)
r.content
# df = pd.read_csv(StringIO(r.content.decode('utf-8')))

b'Candidate/Committee,Committee Type,CanCommOffice,CanCommDistrict,CanCommJurisdiction,CanCommParty,CanCommIncumbentStatus,CanCommFinancingStatus,Date,Contributor,Purpose/Type,Amount,Employer,Occupation,OccupationOther,Address,Description,Report Name,Date Filed,Amended,24-Hour Report,Contributor Type,Election Year,Election Type\r\n'

In [64]:
#GET SPENDING REPORTS#

#Set index
x=0
for b in trans_types:
    
    data.update({"TransactionType":trans_types[x]})
    
    dfs=[]
    
    #Reset index
    i=0
    for n in comm_types:

        data.update({"CommitteeType":comm_types[i]})

        r = s.post(url, data=json.dumps(data), headers=headers)
        df = pd.read_csv(StringIO(r.content.decode('utf-8')))
        dfs.append(df)
        
        i+=1
        
    #Combine dfs
    df=pd.concat(dfs,sort=False,ignore_index=True).drop_duplicates()
    
    count = dw.query('darrenfishell/2020-maine-state-campaign-finance', 'SELECT COUNT(*) FROM '+filenames[x]).dataframe['count'][0]
    if count < len(df):
        print('Writing file '+filenames[x]+'.csv to data.world, at '+str(len(df))+' records.')
        with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance',filenames[x]+'.csv') as w:
            df.to_csv(w,index=False)
    
    #Advance to next transaction type
    x+=1

In [60]:
#GET CANDIDATES LOOKUP#
url='https://mainecampaignfinance.com/api///Organization/SearchCandidates'

cand_q = {"ElectionYear":"2020",
 "pageNumber":1,
 "pageSize":2147483647}

r = s.post(url, data=json.dumps(cand_q), headers=headers).json()

df = json_normalize(r)

with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance','candidate_lookup.csv') as w:
    df.to_csv(w,index=False)

In [54]:
#GET QUESTION ONE COMMITTEES#
url='https://mainecampaignfinance.com/api///Organization/SearchAssociatedBallotQuestionCommittees'

vax_dict = {"BallotQuestionTypeId":"322"
            ,"ElectionYear":"2020"
            ,"pageNumber":"1"
            ,"pageSize":'10'
            ,"sortDir":"desc"
            ,"sortedBy":""}

r = s.post(url, data=json.dumps(vax_dict), headers=headers).json()

df = json_normalize(r)

with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance','question-one-committees.csv') as w:
    df.to_csv(w,index=False)

[{'Title': None,
  'BallotQuestionNumber': None,
  'BallotQuestion': None,
  'ElectionYear': None,
  'ElectionName': None,
  'Status': 'Active',
  'CommitteesSupport': None,
  'CommitteesOppose': None,
  'CommitteeName': 'Mainers for Health and Parental Rights',
  'CommitteeType': 'Political Action Committee',
  'TotalToSupport': 130787.82,
  'TotalToOppose': 0.0,
  'ElectionId': None,
  'CommitteeId': 353083,
  'SupportFlagStatus': 'Support',
  'RowNumber': None,
  'TotalRows': 5,
  'BallotQuestionTypeId': None,
  'ActiveFlag': 0},
 {'Title': None,
  'BallotQuestionNumber': None,
  'BallotQuestion': None,
  'ElectionYear': None,
  'ElectionName': None,
  'Status': 'Active',
  'CommitteesSupport': None,
  'CommitteesOppose': None,
  'CommitteeName': "DON'T TREAD ON MAINE PAC",
  'CommitteeType': 'Political Action Committee',
  'TotalToSupport': 100.0,
  'TotalToOppose': 0.0,
  'ElectionId': None,
  'CommitteeId': 354202,
  'SupportFlagStatus': 'Support',
  'RowNumber': None,
  'TotalRo

In [31]:
# ##Loop through filenames##
# with dw.open_remote_file('darrenfishell/2020-maine-state-campaign-finance',filenames[i]) as w:
#         df.to_csv(w,index=False)